<a href="https://colab.research.google.com/github/nitish-pandey/Spam-Classification/blob/main/Spam_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Basic Imports

In [ ]:
from os import remove
import pandas as pd
import numpy 
import sklearn
import re

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score,roc_auc_score,f1_score,log_loss,precision_score,recall_score,classification_report

from sklearn.naive_bayes import GaussianNB,MultinomialNB

# Data imports and preprocessing

In [ ]:
dataset=pd.read_csv(r'/content/spam.csv',encoding='latin-1')
dataset.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [ ]:
X=dataset['v2']
y=dataset['v1']

y=pd.get_dummies(y,drop_first=True)
print(y.head(),'\n\n',X.head())

   spam
0     0
1     0
2     1
3     0
4     0 

 0    Go until jurong point, crazy.. Available only ...
1                        Ok lar... Joking wif u oni...
2    Free entry in 2 a wkly comp to win FA Cup fina...
3    U dun say so early hor... U c already then say...
4    Nah I don't think he goes to usf, he lives aro...
Name: v2, dtype: object


# Data Cleaning

In [ ]:
stopword=set(stopwords.words('english'))

lemmatizer=WordNetLemmatizer()

n,=X.shape
n

5572

In [ ]:
for i in range(n):
    temp=re.sub('[^a-zA-Z]',' ',X[i])
    temp=temp.lower()
    temp=temp.split()
    temp=[lemmatizer.lemmatize(word) for word in temp if word not in set(stopword)]
    temp=' '.join(temp)
    X[i]=temp


In [ ]:
cv=CountVectorizer()

X=cv.fit_transform(X).toarray()

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)

# Machine learning implementation

In [ ]:
model=MultinomialNB()
model.fit(X_train,y_train.values.ravel())

MultinomialNB()

In [ ]:
y_pred = model.predict(X_test)
print('Classification Report : \n',classification_report(y_test,y_pred))
print('\nConfusion Matrix\n',confusion_matrix(y_test,y_pred))

Classification Report : 
               precision    recall  f1-score   support

           0       0.99      0.98      0.99      1450
           1       0.88      0.96      0.92       222

    accuracy                           0.98      1672
   macro avg       0.94      0.97      0.95      1672
weighted avg       0.98      0.98      0.98      1672


Confusion Matrix
 [[1420   30]
 [   9  213]]


# Saving the Model

In [ ]:
import joblib

joblib.dump(model,'mymodel.pkl')
joblib.dump(cv,'countvector.pkl')

['countvector.pkl']